In [71]:
from util import load_data

name = 'yelp_reviews'

dataframes, relationships = load_data()
target_entity = "review"

entity_col = "review_id"
time_col = "date"

window_size = "1m"
minimum_data = dataframes["review"][0]["date"].min()
maximum_data = dataframes["review"][0]["date"].max()

### Load data

In [72]:
from trane.parsing import denormalize
import pandas as pd

df = denormalize(
    dataframes,
    relationships,
    target_entity
)
df = df.drop(["business.categories", "business.neighborhoods"], axis=1)
df = df.dropna()

## Trane

#### Generate prediction problems

In [73]:
from trane import CutoffStrategy
from util import get_meta

meta = get_meta(df, entity_col)

cutoff_strategy = CutoffStrategy(
    entity_col=entity_col,
    window_size=window_size,
    minimum_data=minimum_data,
    maximum_data=maximum_data,
)

from trane import PredictionProblemGenerator
problem_generator = PredictionProblemGenerator(
    df=df,
    table_meta=meta,
    entity_col=entity_col,
    cutoff_strategy=cutoff_strategy,
    time_col=time_col,
)
problems = problem_generator.generate(df, generate_thresholds=True)

  0%|          | 0/895907 [00:00<?, ?it/s]

Find a relevant problem

In [ ]:
with open("generated_problems.txt", "w") as text_file:
    for idx, p in enumerate(problems):
        print(idx, p, file=text_file)

In [ ]:
ex = problems[3781].execute(df, num_examples_per_instance=-1)
problems[3781]

#### Generate features

In [ ]:
from featuretools_util import get_features

X, y = get_features(
    name=name,
    df=df,
    target_entity=target_entity,
    entity_col=entity_col,
    time_index=time_col,
    relationships=relationships,
    cutoff_time=ex
)

X.head()